In [ ]:
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import asyncio
import nest_asyncio

nest_asyncio.apply()

SEZON = 2019 #dowolne sezony

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')  
            driver = webdriver.Chrome(options=options)
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
            html = driver.page_source
            driver.quit()
            soup = BeautifulSoup(html, 'html.parser')
            element = soup.select_one(selector)
            if element:
                html = str(element)
                break
        except TimeoutException:
            print(f"Timeout error on {url}")
        finally:
            driver.quit()
    return html

async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all("a")
        standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]

        for url in standings_pages:
            save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
            if os.path.exists(save_path):
                continue

            html = await get_html(url, "#all_schedule")
            if html:
                with open(save_path, "w+", encoding='utf-8') as f:
                    f.write(html)
                print(f"Zapisano plik: {save_path}")

async def scrape_game(standings_file):
    with open(standings_file, 'r', encoding='utf-8', errors='ignore') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+", encoding='utf-8') as f:
            f.write(html)
        print(f"Zapisano plik: {save_path}")

async def main():
    await scrape_season(SEZON)
    
    standings_files = [f for f in os.listdir(STANDINGS_DIR) if str(SEZON) in f]
    for standings_file in standings_files:
        filepath = os.path.join(STANDINGS_DIR, standings_file)
        await scrape_game(filepath)

if __name__ == "__main__":
    asyncio.run(main())